## Autoencoder für die Reihenfolge:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

mpl.rcParams['figure.figsize'] = (15, 5)

np.random.seed(42)

### Input Data:
Das 3D Array, das alle latent_representations der Wavesets für jeden Song enthält.

In [7]:
with open("data/song_orders.txt", "rb") as fp:
    song_orders = pickle.load(fp)

print(song_orders)
arr = np.dstack(song_orders)

[[[12.68275, -2.3526], [198.431, 67.43]], [[-4.896, -69.123]]]


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2 and the array at index 1 has size 1

### Encoder: